In [1]:
import pandas as pd
import numpy as np

#pip install pandas-profiling
# pip install scikit-plot
#pip install missingno

from sklearn.preprocessing import Imputer
%matplotlib inline

import math
import gc 
pd.options.display.max_columns = 99

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [3]:
data_cleaned = pd.read_csv('../tmp/data_cleaned.csv')

In [4]:
target = data_cleaned['target']
data_cleaned.drop('target',axis=1,inplace=True)

#Missing values
data_missing = (data_cleaned.isnull().sum() / len(data_cleaned)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.75]
all_missing = list(set(set(data_missing)))
print('There are %d columns with more than 75%% missing values' % len(all_missing))
data_cleaned.drop(columns = all_missing,inplace=True)


#Corrélation
corr_matrix = data_cleaned.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
data_cleaned.drop(columns=to_drop,axis=1,inplace=True)



print("Final shape")
print(data_cleaned.shape)

data_cleaned['target'] = target

test = data_cleaned[data_cleaned.target.isnull()]
train = data_cleaned[data_cleaned.target.notnull()]

There are 28 columns with more than 75% missing values
Final shape
(356255, 588)


In [5]:
train.to_csv("../tmp/train.csv",index=False)
test.to_csv("../tmp/test.csv",index=False)